In [1]:
import warnings
from src.utils_functions import *

# Display
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format
sns.set_theme(context='notebook', style=plt.style.use('dark_background'))
warnings.simplefilter('ignore')

%load_ext autoreload
%autoreload 2

In [98]:
# Carga y lectura de los datos
ipc = pd.read_csv('datasets/interim/data_interim_mensual.csv',
                   usecols=['Fecha', 'Índice', 'Mensual'],
                   parse_dates=True,
                   index_col=[0])
ipc.columns = ['IPC', 'Retornos']

# Definimos la frecuencia de nuestras series temporales
ipc = ipc.asfreq('MS')
ipc

,IPC,Retornos
Fecha,,
2013-02-01,102.19,1.04
2013-03-01,102.43,0.22
2013-04-01,102.82,0.37
2013-05-01,103.38,0.54
2013-06-01,104.73,1.30
...,...,...
2023-03-01,170.80,0.26
2023-04-01,171.95,0.04
2023-05-01,172.03,0.29


In [94]:
# Tasa de interés
interest_rate = pd.read_csv('./datasets/raw/Tasa_de_interes_India.csv',
                            index_col=[0],
                            parse_dates=True)
interest_rate.columns = ['Interest Rate']
interest_rate = interest_rate.asfreq('MS')
interest_rate

,Interest Rate
DATE,
2013-02-01,7.84
2013-03-01,7.91
2013-04-01,7.83
2013-05-01,7.50
2013-06-01,7.40
...,...
2023-03-01,7.36
2023-04-01,7.18
2023-05-01,7.01


In [96]:
# Oferta Monetaria
monetary_aggregate = pd.read_csv('./datasets/raw/Oferta_Monetaria_India.csv',
                               index_col=[0],
                               parse_dates=True)
monetary_aggregate.columns = ['Monetary Aggregate']
monetary_aggregate = monetary_aggregate.asfreq('MS')
monetary_aggregate

,Monetary Aggregate
DATE,
2013-02-01,"18,454,240,000,545.30"
2013-03-01,"18,529,904,293,813.70"
2013-04-01,"18,641,943,582,120.00"
2013-05-01,"18,889,355,664,266.60"
2013-06-01,"19,534,130,565,330.50"
...,...
2023-03-01,"54,640,281,148,084.31"
2023-04-01,"55,090,435,223,493.40"
2023-05-01,"56,926,846,365,222.00"


In [97]:
# Tasa de cambio
exchange_rate = pd.read_csv('./datasets/raw/Tipo_de_cambio_India.csv',
                            index_col=[0],
                            parse_dates=True)
exchange_rate.columns = ['Exchange Rate']
exchange_rate = exchange_rate.asfreq('MS')
exchange_rate

,Exchange Rate
DATE,
2013-02-01,53.81
2013-03-01,54.42
2013-04-01,54.32
2013-05-01,54.98
2013-06-01,58.38
...,...
2023-03-01,82.27
2023-04-01,81.97
2023-05-01,82.30


### *Test de Causalidad de Granger*

* Hipótesis Nula $(H_0)$: La hipótesis nula en este test establece que no hay causalidad Granger entre las dos series de tiempo. En otras palabras, los valores pasados de una serie de tiempo no tienen un efecto estadísticamente significativo en la predicción de la otra serie.

* Hipótesis Alternativa $(H_1)$: La hipótesis alternativa sugiere que sí existe causalidad Granger entre las dos series de tiempo. Esto significa que los valores pasados de una serie de tiempo tienen un efecto estadísticamente significativo en la predicción de la otra serie.

El objetivo del **Test de Causalidad de Granger** es determinar si hay evidencia estadística para rechazar la hipótesis nula en favor de la hipótesis alternativa. En otras palabras, si el p-valor obtenido en el test es menor que un nivel de significancia predefinido (por ejemplo, 0.05), entonces se rechaza la hipótesis nula y se concluye que existe causalidad Granger. Por otro lado, si el p-valor es mayor que el nivel de significancia, no se rechaza la hipótesis nula, lo que indica que no hay evidencia suficiente para afirmar que existe causalidad Granger.

In [99]:
# Concatenamos todas las series
data = pd.concat([ipc['IPC'], interest_rate, monetary_aggregate, exchange_rate], axis=1)
data

,IPC,Interest Rate,Monetary Aggregate,Exchange Rate
2013-02-01,102.19,7.84,"18,454,240,000,545.30",53.81
2013-03-01,102.43,7.91,"18,529,904,293,813.70",54.42
2013-04-01,102.82,7.83,"18,641,943,582,120.00",54.32
2013-05-01,103.38,7.50,"18,889,355,664,266.60",54.98
2013-06-01,104.73,7.40,"19,534,130,565,330.50",58.38
...,...,...,...,...
2023-03-01,170.80,7.36,"54,640,281,148,084.31",82.27
2023-04-01,171.95,7.18,"55,090,435,223,493.40",81.97
2023-05-01,172.03,7.01,"56,926,846,365,222.00",82.30
2023-06-01,172.47,7.05,"58,650,795,417,338.00",82.23


In [103]:
# Transformarmos las series a estacionarias
series_estacionarias, derivadas = get_stationarity(data)
series_estacionarias

,IPC,Interest Rate,Monetary Aggregate,Exchange Rate
2013-03-01,0.24,0.07,"75,664,293,268.40",0.62
2013-04-01,0.39,-0.08,"112,039,288,306.30",-0.10
2013-05-01,0.57,-0.33,"247,412,082,146.60",0.66
2013-06-01,1.35,-0.10,"644,774,901,063.90",3.40
2013-07-01,0.38,0.32,"-320,542,351,785.70",1.38
...,...,...,...,...
2023-03-01,-0.01,0.02,"-1,099,491,481,485.59",-0.33
2023-04-01,1.15,-0.18,"450,154,075,409.09",-0.30
2023-05-01,0.08,-0.17,"1,836,411,141,728.60",0.33
2023-06-01,0.44,0.04,"1,723,949,052,116.00",-0.07


In [112]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag = 15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test=test, verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

granger_results = grangers_causation_matrix(data=series_estacionarias,
                                            variables=series_estacionarias.columns)

In [113]:
granger_results

,IPC_x,Interest Rate_x,Monetary Aggregate_x,Exchange Rate_x
IPC_y,1.00,0.00,0.37,0.01
Interest Rate_y,0.09,1.00,0.03,0.08
Monetary Aggregate_y,0.67,0.58,1.00,0.02
Exchange Rate_y,0.04,0.01,0.19,1.00


In [109]:
# Interpretación: Si un p-valor mínimo es más grande (por encima de 0.05), 
# no hay suficiente evidencia para afirmar la causalidad entre las dos series en esa fila y columna.

print("""Resultados:
• Entre "IPC" y "Interest Rate", el p-valor mínimo es muy pequeño (0.00), lo que sugiere que "IPC" causa "Interest Rate."
• Entre "Interest Rate" y "Monetary Aggregate", el p-valor mínimo es pequeño (0.03), lo que sugiere que "Interest Rate" causa "Monetary Aggregate."
• Entre "Monetary Aggregate" y "Exchange Rate", el p-valor mínimo es pequeño (0.02), lo que sugiere que "Monetary Aggregate" causa "Exchange Rate."
• Entre "Exchange Rate" y "IPC", el p-valor mínimo es pequeño (0.01), lo que sugiere que "Exchange Rate" causa "IPC."
""")

Resultados:
• Entre "IPC" y "Interest Rate", el p-valor mínimo es muy pequeño (0.00), lo que sugiere que "IPC" causa "Interest Rate."
• Entre "Interest Rate" y "Monetary Aggregate", el p-valor mínimo es pequeño (0.03), lo que sugiere que "Interest Rate" causa "Monetary Aggregate."
• Entre "Monetary Aggregate" y "Exchange Rate", el p-valor mínimo es pequeño (0.02), lo que sugiere que "Monetary Aggregate" causa "Exchange Rate."
• Entre "Exchange Rate" y "IPC", el p-valor mínimo es pequeño (0.01), lo que sugiere que "Exchange Rate" causa "IPC."

